In [ ]:
import requests
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

In [ ]:
load_dotenv()
api_key = os.getenv("HOLDERSCAN_API_KEY")

In [ ]:
def get_time_stamp():
    return datetime.now().strftime('%Y%m%d%H%M%S')

In [ ]:
directory = 'holderscan-' + get_time_stamp()
os.makedirs(directory, exist_ok=True)

In [ ]:
base_url = 'https://api.holderscan.com/v0'
headers = {'x-api-key : ' + api_key}

In [ ]:
def get_holder_breakdown(ca) :
    headers = {'x-api-key :' + api_key}
    return requests.get(base_url + '/sol/tokens/' + ca + '/holders/breakdowns', headers=headers).json()

In [ ]:
ca_example = ''
holder_breakdown = get_holder_breakdown(api_key)
